In [1]:
from transformers import pipeline
import numpy as np

# Initialize
emotion_classifier = pipeline(
    "text-classification",
    model="SamLowe/roberta-base-go_emotions",
    top_k=None  # Returns all 28 emotions with scores
)

def analyze_adult_turn(text):
    """
    Analyze adult turn and extract pedagogically-relevant scores
    """
    # Get all emotion scores
    emotions = emotion_classifier(text)[0]
    
    # Convert to dict for easier access
    emotion_dict = {e['label']: e['score'] for e in emotions}
    
    # Compute composite scores
    scores = {
        # 1. SUPPORTIVENESS: How encouraging/warm is the adult?
        'supportiveness': sum([
            emotion_dict.get('admiration', 0),
            emotion_dict.get('approval', 0),
            emotion_dict.get('caring', 0),
            emotion_dict.get('optimism', 0),
            emotion_dict.get('pride', 0),
        ]) / 5,
        
        # 2. ENGAGEMENT: Is the adult actively engaged/interested?
        'engagement': sum([
            emotion_dict.get('curiosity', 0),
            emotion_dict.get('excitement', 0),
            emotion_dict.get('amusement', 0),
            emotion_dict.get('surprise', 0),
        ]) / 4,
        
        # 3. WARMTH: Emotional connection
        'warmth': sum([
            emotion_dict.get('caring', 0),
            emotion_dict.get('love', 0),
            emotion_dict.get('gratitude', 0),
            emotion_dict.get('joy', 0),
        ]) / 4,
        
        # 4. NEGATIVITY: Potentially discouraging (should be low)
        'negativity': sum([
            emotion_dict.get('disappointment', 0),
            emotion_dict.get('disapproval', 0),
            emotion_dict.get('annoyance', 0),
            emotion_dict.get('anger', 0),
        ]) / 4,
        
        # 5. Individual emotions you care about most
        'approval': emotion_dict.get('approval', 0),
        'curiosity': emotion_dict.get('curiosity', 0),
        'caring': emotion_dict.get('caring', 0),
        'neutral': emotion_dict.get('neutral', 0),
        
        # Raw distribution for detailed analysis
        'full_distribution': emotion_dict
    }
    
    return scores


/Users/jliu/anaconda3/envs/genai/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use mps:0



Text: That's exactly right! I'm so proud of how you figured that out!
Supportiveness: 0.274
Engagement: 0.013
Warmth: 0.018
Negativity: 0.011

Text: Hmm, not quite. Let's think about this differently.
Supportiveness: 0.014
Engagement: 0.004
Warmth: 0.002
Negativity: 0.132

Text: You're not listening. Pay attention.
Supportiveness: 0.043
Engagement: 0.001
Warmth: 0.043
Negativity: 0.172

Text: Wow, what an interesting idea! Tell me more about your thinking.
Supportiveness: 0.096
Engagement: 0.210
Warmth: 0.007
Negativity: 0.002


In [ ]:

# Example usage
examples = [
    "That's exactly right! I'm so proud of how you figured that out!",  # High supportiveness
    "Hmm, not quite. Let's think about this differently.",              # Neutral/corrective
    "You're not listening. Pay attention.",                             # Low supportiveness
    "Wow, what an interesting idea! Tell me more about your thinking.", # High engagement
]

for text in examples:
    scores = analyze_adult_turn(text)
    print(f"\nText: {text}")
    print(f"Supportiveness: {scores['supportiveness']:.3f}")
    print(f"Engagement: {scores['engagement']:.3f}")
    print(f"Warmth: {scores['warmth']:.3f}")
    print(f"Negativity: {scores['negativity']:.3f}")

In [2]:
scores

{'supportiveness': 0.09580207278486341,
 'engagement': 0.20974237541668117,
 'warmth': 0.007456185179762542,
 'negativity': 0.0021727741113863885,
 'approval': 0.035716451704502106,
 'curiosity': 0.33084070682525635,
 'caring': 0.0024520084261894226,
 'neutral': 0.017057642340660095,
 'full_distribution': {'admiration': 0.431988924741745,
  'curiosity': 0.33084070682525635,
  'excitement': 0.31234994530677795,
  'surprise': 0.18875785171985626,
  'approval': 0.035716451704502106,
  'neutral': 0.017057642340660095,
  'joy': 0.01614595577120781,
  'confusion': 0.013953295536339283,
  'realization': 0.009278860874474049,
  'optimism': 0.0074264779686927795,
  'amusement': 0.007020997814834118,
  'love': 0.006129618734121323,
  'gratitude': 0.005097157787531614,
  'desire': 0.0034482087939977646,
  'annoyance': 0.0029239472933113575,
  'disapproval': 0.0026616048999130726,
  'caring': 0.0024520084261894226,
  'fear': 0.002437618328258395,
  'disappointment': 0.0017462191171944141,
  'disgu